In [24]:
from ultralytics import YOLO
from pymongo import MongoClient
import cv2, os
from datetime import datetime

# MongoDB connection
client = MongoClient("mongodb://localhost:27017/")
col    = client["new_animal_db"]["detections"]

# Load YOLOv11n model
model = YOLO(r"C:\Users\Hp\runs\detect\animal_yolov11n_data_1\weights\best.pt")

# Output folder
output_dir = r"F:\animal-results"
os.makedirs(output_dir, exist_ok=True)


In [25]:
# ----- IMAGE PATH -----
img_path = r"F:\animal-dataset\img2.jpg"

img       = cv2.imread(img_path)
results   = model(img, conf=0.25)
annotated = results[0].plot()

out_path  = os.path.join(output_dir, "detected_" + os.path.basename(img_path))
cv2.imwrite(out_path, annotated)
print(f"✅ Image saved to {out_path}")

classes = [model.names[int(c)] for c in results[0].boxes.cls]
confs   = [float(c) for c in results[0].boxes.conf]
col.insert_one({
    "type": "image",
    "image_name": os.path.basename(img_path),
    "detected_animals": classes,
    "confidences": confs,
    "timestamp": datetime.now()
})
print("✅ Metadata saved to MongoDB")



0: 640x640 1 leopard, 30.7ms
Speed: 7.4ms preprocess, 30.7ms inference, 4.3ms postprocess per image at shape (1, 3, 640, 640)
✅ Image saved to F:\animal-results\detected_img2.jpg
✅ Metadata saved to MongoDB


In [26]:
# ----- VIDEO PATH -----
vid_path = r"F:\animal-dataset\snake.mp4"

# Prep VideoWriter once (resolution & fps from OpenCV)
cap  = cv2.VideoCapture(vid_path)
fps, w, h = cap.get(5), int(cap.get(3)), int(cap.get(4))
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out_path = os.path.join(output_dir, "detected_" + os.path.basename(vid_path).replace(".mp4", ".avi"))
writer   = cv2.VideoWriter(out_path, fourcc, fps, (w, h))
cap.release()

frame_id = 0
for results in model.predict(source=vid_path,
                             stream=True,
                             conf=0.25,
                             imgsz=640):

    annotated = results.plot()      # BGR frame with boxes
    writer.write(annotated)

    classes = [results.names[int(c)] for c in results.boxes.cls]
    confs   = [float(c) for c in results.boxes.conf]

    col.insert_one({
        "type": "video",
        "video_name": os.path.basename(vid_path),
        "frame_id": frame_id,
        "detected_animals": classes,
        "confidences": confs,
        "timestamp": datetime.now()
    })
    frame_id += 1

writer.release()
print(f"✅ Annotated video saved to {out_path}")



video 1/1 (frame 1/223) F:\animal-dataset\snake.mp4: 384x640 1 snake, 28.2ms
video 1/1 (frame 2/223) F:\animal-dataset\snake.mp4: 384x640 1 snake, 30.0ms
video 1/1 (frame 3/223) F:\animal-dataset\snake.mp4: 384x640 (no detections), 23.3ms
video 1/1 (frame 4/223) F:\animal-dataset\snake.mp4: 384x640 1 snake, 25.4ms
video 1/1 (frame 5/223) F:\animal-dataset\snake.mp4: 384x640 1 snake, 22.9ms
video 1/1 (frame 6/223) F:\animal-dataset\snake.mp4: 384x640 1 snake, 22.3ms
video 1/1 (frame 7/223) F:\animal-dataset\snake.mp4: 384x640 1 snake, 23.5ms
video 1/1 (frame 8/223) F:\animal-dataset\snake.mp4: 384x640 1 snake, 26.4ms
video 1/1 (frame 9/223) F:\animal-dataset\snake.mp4: 384x640 1 snake, 22.3ms
video 1/1 (frame 10/223) F:\animal-dataset\snake.mp4: 384x640 1 snake, 28.4ms
video 1/1 (frame 11/223) F:\animal-dataset\snake.mp4: 384x640 1 snake, 20.0ms
video 1/1 (frame 12/223) F:\animal-dataset\snake.mp4: 384x640 1 snake, 21.0ms
video 1/1 (frame 13/223) F:\animal-dataset\snake.mp4: 384x640 (n